In [1]:
# Template Notebook for Engagement Classifcation Model

# ToDo/Models to run here: https://docs.google.com/document/d/18EJpFesMEl9Q7C1AZzDeq6uy7c8tDMvEJ_j58-MmiBI/edit?usp=sharing
# Use p8_data_processed.csv here: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [2]:
# Imports

import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [3]:
# Load Data
# Warning: this will probably take some time
# Adjust file path based on your computer

file = '../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/p8_data_processed (1).csv'
data = pd.read_csv(file)

/Users/leenamathur/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.head()

,participant,session_num,session_date,timestamp,engagement,activity,skill,difficulty,aptitude,games_total,...,op_p2_pose_y9,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,8.0,1.0,2018-06-07,0.000000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
1,8.0,1.0,2018-06-07,0.033333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
2,8.0,1.0,2018-06-07,0.066667,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
3,8.0,1.0,2018-06-07,0.100000,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1
4,8.0,1.0,2018-06-07,0.133333,1.0,NaN,NaN,NaN,0.5,0.0,...,0.0,0,0,0,0,0,0,0,0,1


In [4]:
# Drop Irrelevant Columns

# transcriptions
data = data.drop(columns=['transcript_spk_0', 'transcript_spk_1', 'transcript_spk_2'])

# raw ros messages
data = data.drop(columns=['ros_PARTICIPANT_STATE', 'ros_ROBOT_STATE'])

# more for counting purposes
data = data.drop(columns=['game_start', 'game_correct', 'game_incorrect', 'mistake_made'])

data = data.drop(columns=['participant', 'session_date'])

# use one-hot encoded difficulty and skill, don't use activity for now
data = data.drop(columns=['difficulty', 'skill', 'activity', 'of_face_id'])

In [5]:
# Open-Face/Open-Pose Columns? 
# Feature Dictionary Documentation here: https://docs.google.com/document/d/1RSygoLwsM1PKEIOoqDOaEo3lRBZBLZJuzbMSY5K_6zA/edit?usp=sharing

only_ofop = []
performance = []
for i in data.columns:
    # open face/open pose columns begin with of_ or op_
    if 'op_' in i or 'of_' in i:
        only_ofop.append(i)
    else:
        performance.append(i)
        
only_ofop.remove('of_timestamp')

In [6]:
# Label Analysis

print(data['engagement'].value_counts())
print()

print(1,data['engagement'].value_counts()[1]/sum(data['engagement'].value_counts()))
print(0,data['engagement'].value_counts()[0]/sum(data['engagement'].value_counts()))

1.0    139463
0.0    120473
Name: engagement, dtype: int64

1 0.5365282223316509
0 0.4634717776683491


### Step 1: Set Up Feature Sets

1. All OF/OP + Performance
2. Performance ONLY
3. All OF/OP ONLY
4. Performance + Handpicked OF/OP (from Caitlyn's intuition)
5. Performance + Selected OF/OP (from Feature Analysis) 

Note: keep timestamp and session_num until right before running the model in all feature sets

In [10]:
# Feature Set 1: All OF/OP + Performance

FS1 = data

In [11]:
# Feature Set 2: Performance ONLY

print(performance)

FS2 = data[performance]

['session_num', 'timestamp', 'engagement', 'aptitude', 'games_total', 'games_session', 'mistakes_total', 'mistakes_session', 'mistakes_game', 'ts_robot_talked', 'ts_game_start', 'ts_attempt', 'skill_NC', 'skill_OS', 'skill_EM', 'diff_1', 'diff_2', 'diff_3', 'diff_4', 'diff_5', 'no_game']


In [7]:
# Feature Set 3: OF/OP ONLY  

only_ofop.append('timestamp')
only_ofop.append('session_num')
only_ofop.append('engagement')

FS3 = data[only_ofop]

In [13]:
# Feature Set 4: Performance + Handpicked OF/OP

hand_picked = ['confidence', 'success', 'gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x', 'gaze_angle_y']
for i,c in enumerate(hand_picked):
    hand_picked[i] = 'of_' + c
hand_picked.append('op_Number of People')

for i in performance:
    hand_picked.append(i)

FS4 = data[hand_picked]

In [14]:
# Feature Set 5: Performance + Selected OF/OP

selected_ofop = ['of_AU04_c',   'of_AU06_c',    'of_AU07_c',    'of_AU14_c',    'of_AU15_c',    'of_AU25_c',    'of_confidence',    'of_frame',    'of_success',    'of_timestamp', 'op_p1_face_c25',    'op_p1_face_c43',    'op_p1_face_c44',    'op_p1_face_c45',    'op_p1_face_c46',    'op_p1_face_y7',    'op_p1_face_y8',    'op_p1_pose_c0',    'op_p1_pose_c15',    'op_p1_pose_c16']

for i in performance:
    selected_ofop.append(i)

FS5 = data[selected_ofop]

### Step 2: Train Test Split (for chosen Feature Set)

1. Random Shuffle 70-30
2. Train on Earlier Sessions, Test on Later Sessions 70-30

In [8]:
# CHOOSE FEATURE SET

FS = FS3

In [9]:
# sort values
FS = FS.sort_values(['session_num', 'timestamp'])

In [10]:
# Train-Test Split 1: Random Shuffle 70-30

y1 = FS['engagement']
X1 = FS.drop(columns=['timestamp', 'session_num', 'engagement'])

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [11]:
# Train Test Split 2: Train on Earlier Sessions, Test on Later Sessions

session_threshold = 8 # train on sessions <= 8, test on sessions > 8

train = FS[(FS['session_num'] <= 8.0)] 
test = FS[(FS['session_num'] > 8)]

# drop columns needed for split
train2 = train.drop(columns=['session_num', 'timestamp'])
test2 = test.drop(columns=['session_num', 'timestamp'])

X_train2 = train2.drop(columns=['engagement'])
y_train2 = train2['engagement']

X_test2 = test2.drop(columns=['engagement'])
y_test2 = test2['engagement']

### Step 3: Implement Your Model!

In [17]:
# For chosen feature set, run on both Train-Test Splits

In [12]:
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
# 3 questions from HMM:
# (1) given model, what is likelihood of sequence S happening?
# (2) given model and sequence S, what is optimal hidden state sequence?
# (3) given sequence S and # of hidden states, what is optimal model to maximize probability of S?

In [13]:
#function to fit the hmm
def fitHMM(D):
    # fit Gaussian HMM to the given Data
    model = GaussianHMM(n_components=2, n_iter=1000, algorithm='viterbi').fit([D])
     
    # classify each observation as state 0 or 1
    hidden_states = model.predict(D)
 
    # find parameters of Gaussian HMM
    mus = np.array(model.means_)
    sigmas = np.array(np.sqrt(np.array([np.diag(model.covars_[0]),np.diag(model.covars_[1])])))
    P = np.array(model.transmat_)
 
    # find log-likelihood of Gaussian HMM
    logProb = model.score(D)
 
    return model, hidden_states, mus, sigmas, P, logProb

In [20]:
list(data)

['session_num',
 'timestamp',
 'engagement',
 'aptitude',
 'games_total',
 'games_session',
 'mistakes_total',
 'mistakes_session',
 'mistakes_game',
 'ts_robot_talked',
 'ts_game_start',
 'ts_attempt',
 'of_AU01_c',
 'of_AU01_r',
 'of_AU02_c',
 'of_AU02_r',
 'of_AU04_c',
 'of_AU04_r',
 'of_AU05_c',
 'of_AU05_r',
 'of_AU06_c',
 'of_AU06_r',
 'of_AU07_c',
 'of_AU07_r',
 'of_AU09_c',
 'of_AU09_r',
 'of_AU10_c',
 'of_AU10_r',
 'of_AU12_c',
 'of_AU12_r',
 'of_AU14_c',
 'of_AU14_r',
 'of_AU15_c',
 'of_AU15_r',
 'of_AU17_c',
 'of_AU17_r',
 'of_AU20_c',
 'of_AU20_r',
 'of_AU23_c',
 'of_AU23_r',
 'of_AU25_c',
 'of_AU25_r',
 'of_AU26_c',
 'of_AU26_r',
 'of_AU28_c',
 'of_AU45_c',
 'of_AU45_r',
 'of_X_0',
 'of_X_1',
 'of_X_10',
 'of_X_11',
 'of_X_12',
 'of_X_13',
 'of_X_14',
 'of_X_15',
 'of_X_16',
 'of_X_17',
 'of_X_18',
 'of_X_19',
 'of_X_2',
 'of_X_20',
 'of_X_21',
 'of_X_22',
 'of_X_23',
 'of_X_24',
 'of_X_25',
 'of_X_26',
 'of_X_27',
 'of_X_28',
 'of_X_29',
 'of_X_3',
 'of_X_30',
 'of_X_31',

#### Feature Set 2

In [39]:
#FS2 Train/Test/Split 1
FS2_matrix_test1 = X_train1.values
FS2_model_1, FS2_hidden_states_1, FS2_mus_1, FS2_sigmas_1, FS2_P_1, FS2_logProb_1 = fitHMM(FS2_matrix_test1)

#FS2 Train/Test/Split 2
FS2_matrix_test2 = X_train2.values
FS2_model_2, FS2_hidden_states_2, FS2_mus_2, FS2_sigmas_2, FS2_P_2, FS2_logProb_2 = fitHMM(FS2_matrix_test2)

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anacond

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

In [66]:
#evaluate FS2 HMM
FS2_pred1 = FS2_model_1.predict(X_test1.values)
scores1 = FS2_model_1.predict_proba(X_test1.values)
scores1 = scores1[:, 1]

print("Accuracy for FS2 Split 1:",accuracy_score(y_test1, FS2_pred1))
print("AUC:",roc_auc_score(y_test1, scores1)) 
print(metrics.confusion_matrix(y_test1, FS2_pred1))
print(metrics.classification_report(y_test1, FS2_pred1))

FS2_pred2 = FS2_model_2.predict(X_test2.values)
scores2 = FS2_model_2.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS2 Split 2:",accuracy_score(y_test2, FS2_pred2))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS2_pred2))
print(metrics.classification_report(y_test2, FS2_pred2))

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/skl

Accuracy for FS2 Split 1: 0.43687564919659916
AUC: 0.42072818872718654
[[26614  9350]
 [34563  7454]]
             precision    recall  f1-score   support

        0.0       0.44      0.74      0.55     35964
        1.0       0.44      0.18      0.25     42017

avg / total       0.44      0.44      0.39     77981



/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)


Accuracy for FS2 Split 2: 0.4426787957218437
AUC: 0.4233188514650743
[[23956 40621]
 [17949 22566]]
             precision    recall  f1-score   support

        0.0       0.57      0.37      0.45     64577
        1.0       0.36      0.56      0.44     40515

avg / total       0.49      0.44      0.44    105092



/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Feature Set 4

In [85]:
#FS4 Train/Test/Split 1
FS4_matrix_test1 = X_train1.values
FS4_model_1, FS4_hidden_states_1, FS4_mus_1, FS4_sigmas_1, FS4_P_1, FS4_logProb_1 = fitHMM(FS4_matrix_test1)

#FS4 Train/Test/Split 2
FS4_matrix_test2 = X_train2.values
FS4_model_2, FS4_hidden_states_2, FS4_mus_2, FS4_sigmas_2, FS4_P_2, FS4_logProb_2 = fitHMM(FS4_matrix_test2)


/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anacond

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-pack

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-pack

In [86]:
#evaluate FS4 HMM
FS4_pred1 = FS4_model_1.predict(X_test1.values)
scores1 = FS4_model_1.predict_proba(X_test1.values)
scores1 = scores1[:, 1]

print("Accuracy for FS4 Split 1:",accuracy_score(y_test1, FS4_pred1))
print("AUC:",roc_auc_score(y_test1, scores1)) 
print(metrics.confusion_matrix(y_test1, FS4_pred1))
print(metrics.classification_report(y_test1, FS4_pred1))

FS4_pred2 = FS4_model_2.predict(X_test2.values)
scores2 = FS4_model_2.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS4 Split 2:",accuracy_score(y_test2, FS4_pred2))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred2))
print(metrics.classification_report(y_test2, FS4_pred2))

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/skl

Accuracy for FS4 Split 1: 0.34650748259191344
AUC: 0.33460097099847413
[[ 6517 29447]
 [21513 20504]]
             precision    recall  f1-score   support

        0.0       0.23      0.18      0.20     35964
        1.0       0.41      0.49      0.45     42017

avg / total       0.33      0.35      0.33     77981



/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)


Accuracy for FS4 Split 2: 0.4222490769992007
AUC: 0.3345202044497205
[[22973 41604]
 [19113 21402]]
             precision    recall  f1-score   support

        0.0       0.55      0.36      0.43     64577
        1.0       0.34      0.53      0.41     40515

avg / total       0.47      0.42      0.42    105092



/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Feature Set 5

In [91]:
#FS5 Train/Test/Split 1
FS5_matrix_test1 = X_train1.values
FS5_model_1, FS5_hidden_states_1, FS5_mus_1, FS5_sigmas_1, FS5_P_1, FS5_logProb_1 = fitHMM(FS5_matrix_test1)

#FS5 Train/Test/Split 2
FS5_matrix_test2 = X_train2.values
FS5_model_2, FS5_hidden_states_2, FS5_mus_2, FS5_sigmas_2, FS5_P_2, FS5_logProb_2 = fitHMM(FS5_matrix_test2)

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anacond

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/s

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-pack

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-pack

In [92]:
#evaluate FS5 HMM
FS5_pred1 = FS5_model_1.predict(X_test1.values)
scores1 = FS5_model_1.predict_proba(X_test1.values)
scores1 = scores1[:, 1]

print("Accuracy for FS5 Split 1:",accuracy_score(y_test1, FS5_pred1))
print("AUC:",roc_auc_score(y_test1, scores1)) 
print(metrics.confusion_matrix(y_test1, FS5_pred1))
print(metrics.classification_report(y_test1, FS5_pred1))

FS5_pred2 = FS5_model_2.predict(X_test2.values)
scores2 = FS5_model_2.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS5 Split 2:",accuracy_score(y_test2, FS5_pred2))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS5_pred2))
print(metrics.classification_report(y_test2, FS5_pred2))

/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/skl

Accuracy for FS5 Split 1: 0.7135456072633077
AUC: 0.7744922225459866
[[31075  4889]
 [17449 24568]]
             precision    recall  f1-score   support

        0.0       0.64      0.86      0.74     35964
        1.0       0.83      0.58      0.69     42017

avg / total       0.74      0.71      0.71     77981



/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)


Accuracy for FS5 Split 2: 0.3400924903893731
AUC: 0.38097281044040365
[[12325 52252]
 [17099 23416]]
             precision    recall  f1-score   support

        0.0       0.42      0.19      0.26     64577
        1.0       0.31      0.58      0.40     40515

avg / total       0.38      0.34      0.32    105092



/Users/leenamathur/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function logsumexp is deprecated; sklearn.utils.extmath.logsumexp was deprecated in version 0.19 and will be removed in 0.21. Use scipy.misc.logsumexp instead.
  warnings.warn(msg, category=DeprecationWarning)


#### Feature Set 1

In [ ]:
#FS1 Train/Test/Split 1
FS1_matrix_test1 = X_train1.values
FS1_model_1, FS1_hidden_states_1, FS1_mus_1, FS1_sigmas_1, FS1_P_1, FS1_logProb_1 = fitHMM(FS1_matrix_test1)

#FS1 Train/Test/Split 2
FS1_matrix_test2 = X_train2.values
FS1_model_2, FS1_hidden_states_2, FS1_mus_2, FS1_sigmas_2, FS1_P_2, FS1_logProb_2 = fitHMM(FS1_matrix_test2)

In [ ]:
#evaluate FS1 HMM
FS1_pred1 = FS1_model_1.predict(X_test1.values)
scores1 = FS1_model_1.predict_proba(X_test1.values)
scores1 = scores1[:, 1]

print("Accuracy for FS1 Split 1:",accuracy_score(y_test1, FS1_pred1))
print("AUC:",roc_auc_score(y_test1, scores1)) 
print(metrics.confusion_matrix(y_test1, FS1_pred1))
print(metrics.classification_report(y_test1, FS1_pred1))

FS1_pred2 = FS1_model_2.predict(X_test2.values)
scores2 = FS1_model_2.predict_proba(X_test2.values)
scores2 = scores2[: ,1]
print("Accuracy for FS1 Split 2:",accuracy_score(y_test2, FS1_pred2))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS1_pred2))
print(metrics.classification_report(y_test2, FS1_pred2))

#### Feature Set 3

In [ ]:
#FS3 Train/Test/Split 1
FS3_matrix_test1 = X_train1.values
FS3_model_1 = GaussianHMM(n_components=2, n_iter=1000, algorithm='viterbi').fit([FS3_matrix_test1])

#FS3 Train/Test/Split 2
FS3_matrix_test2 = X_train2.values
FS3_model_2 = GaussianHMM(n_components=2, n_iter=1000, algorithm='viterbi').fit([FS3_matrix_test2])